# Analiza Wyszukiwania Semantycznego

Ten notatnik analizuje działanie embeddingów i porównuje odległość cosinusową oraz euklidesową na bazie wiedzy.

In [2]:
%pip install openai python-dotenv scipy pandas
# OpenAI Settings

import os
import sys

# Add current directory to path to allow importing knowledge_base.py
sys.path.append(os.getcwd())

from dotenv import load_dotenv

load_dotenv()

APIKey = os.getenv("OPENAI_API_KEY")

import openai
from openai import OpenAI

client = OpenAI(
    api_key=APIKey,
)


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Ładowanie Bazy Wiedzy

In [3]:
# Knowledge base
# Importing the custom knowledge base with 50+ entries
try:
    from zadanie2.knowledge_base import knowledge_base
except ImportError:
    # Fallback if running from root directory
    from knowledge_base import knowledge_base

print(f"Loaded knowledge base with {len(knowledge_base)} entries.")

Loaded knowledge base with 52 entries.


In [4]:
# function to generate embeddings 
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

## Generowanie Embeddingów

In [5]:
# Create a database 
import pandas as pd

# Generate embeddings for the knowledge base
# This might take a moment for 50+ items
print("Generating embeddings...")
embeddings = [get_embedding(text) for text in knowledge_base]

df = pd.DataFrame({
    'Text': knowledge_base,
    'Vector': embeddings
})

print("Embeddings generated.")
df.head()

Generating embeddings...
Embeddings generated.


,Text,Vector
0,1. Dostęp do Głównego Centrum Sterowania mają ...,"[-0.006379794795066118, -0.007359823677688837,..."
1,2. Wszyscy członkowie załogi muszą nosić ident...,"[-0.007427488453686237, 0.0031164397951215506,..."
2,3. Śluza powietrzna nr 4 jest wyłączona z użyt...,"[-0.008868474513292313, 0.009164949879050255, ..."
3,"4. W przypadku wykrycia spadku ciśnienia, mask...","[-0.01356863509863615, 0.012425138615071774, 0..."
4,5. Konsumpcja racji żywnościowych poza mesą je...,"[0.015307189896702766, 0.005199384409934282, -..."


## Testowanie Zapytań i Analiza Odległości

In [6]:
## Queries
# Using a query relevant to the Space Station Omega theme
query = "Jakie są procedury w przypadku pożaru?"
print(f"Query: {query}")

query_vector = get_embedding(query)

Query: Jakie są procedury w przypadku pożaru?


In [7]:
# 1. Cosine Similarity (original method)
from scipy.spatial.distance import cosine

# Note: cosine distance in scipy is 1 - cosine_similarity. 
# So smaller value means closer (more similar).
df['Cosine_Distance'] = df['Vector'].apply(lambda x: cosine(x, query_vector))

print("\n--- Cosine Distance Results (Top 3) ---")
cosine_results = df.nsmallest(3, 'Cosine_Distance')
for i, row in cosine_results.iterrows():
    print(f"Dist: {row['Cosine_Distance']:.4f} | Text: {row['Text']}")


--- Cosine Distance Results (Top 3) ---
Dist: 0.1422 | Text: 29. W przypadku pożaru, systemy Halonowe w danej sekcji aktywują się po 10 sekundach od wykrycia dymu.
Dist: 0.1846 | Text: 30. Procedura 'Last Hope' zakłada ewakuację całej załogi do kapsuł ratunkowych w ciągu 15 minut.
Dist: 0.1869 | Text: 28. Alarm Niebieski oznacza awarię medyczną lub wypadek przy pracy wymagający zespołu ratunkowego.


In [8]:
# 2. Euclidean Distance (new method)
from scipy.spatial.distance import euclidean

df['Euclidean_Distance'] = df['Vector'].apply(lambda x: euclidean(x, query_vector))

print("\n--- Euclidean Distance Results (Top 3) ---")
euclidean_results = df.nsmallest(3, 'Euclidean_Distance')
for i, row in euclidean_results.iterrows():
    print(f"Dist: {row['Euclidean_Distance']:.4f} | Text: {row['Text']}")


--- Euclidean Distance Results (Top 3) ---
Dist: 0.5332 | Text: 29. W przypadku pożaru, systemy Halonowe w danej sekcji aktywują się po 10 sekundach od wykrycia dymu.
Dist: 0.6076 | Text: 30. Procedura 'Last Hope' zakłada ewakuację całej załogi do kapsuł ratunkowych w ciągu 15 minut.
Dist: 0.6114 | Text: 28. Alarm Niebieski oznacza awarię medyczną lub wypadek przy pracy wymagający zespołu ratunkowego.


In [9]:
# Comparison
print("\n--- Comparison ---")
cosine_top_indices = cosine_results.index.tolist()
euclidean_top_indices = euclidean_results.index.tolist()

if cosine_top_indices == euclidean_top_indices:
    print("Both metrics returned the same top 3 results.")
else:
    print("The metrics returned different results.")
    print(f"Cosine Indices: {cosine_top_indices}")
    print(f"Euclidean Indices: {euclidean_top_indices}")


--- Comparison ---
Both metrics returned the same top 3 results.
